In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install easyocr
import os
import cv2
import re
import numpy as np
from tqdm import tqdm
import easyocr
import xml.etree.ElementTree as ET




In [3]:
BASE_DATASET = "/content/drive/MyDrive/kaggle_datasets/indian_vehicle_dataset"
IMG_DIR = os.path.join(BASE_DATASET, "all_images")
XML_DIR = os.path.join(BASE_DATASET, "all_xml")

print("Images:", len(os.listdir(IMG_DIR)))
print("XMLs  :", len(os.listdir(XML_DIR)))


Images: 1339
XMLs  : 1338


In [4]:
def normalize_text(text):
    if text is None:
        return ""
    text = text.upper()
    text = re.sub(r"[^A-Z0-9]", "", text)
    return text


In [5]:
def parse_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    filename = root.find("filename").text.strip()

    obj = root.find("object")
    if obj is None:
        return None

    name_tag = obj.find("name")
    bnd = obj.find("bndbox")

    if name_tag is None or bnd is None:
        return None

    plate_text = name_tag.text.strip()

    bbox = [
        int(bnd.find("xmin").text),
        int(bnd.find("ymin").text),
        int(bnd.find("xmax").text),
        int(bnd.find("ymax").text),
    ]

    return {
        "filename": filename,
        "bbox": bbox,
        "plate_text": normalize_text(plate_text)
    }


In [6]:
gt_data = {}

for xml_file in os.listdir(XML_DIR):
    if not xml_file.endswith(".xml"):
        continue

    parsed = parse_xml(os.path.join(XML_DIR, xml_file))
    if parsed is None:
        continue

    # image MUST exist
    img_path = os.path.join(IMG_DIR, parsed["filename"])
    if not os.path.exists(img_path):
        continue

    gt_data[parsed["filename"]] = {
        "bbox": parsed["bbox"],
        "plate_text": parsed["plate_text"]
    }

print("✅ GT samples loaded:", len(gt_data))


✅ GT samples loaded: 1337


In [7]:
reader = easyocr.Reader(['en'], gpu=True)

def crop_plate(image, bbox):
    x1,y1,x2,y2 = bbox
    return image[y1:y2, x1:x2]


In [8]:
import torch
print(torch.cuda.is_available())

False


In [12]:
print(f"EasyOCR is running on device: {reader.device}")

EasyOCR is running on device: cpu


In [9]:
cer_values = []
exact = 0

def cer(gt, pred):
    if len(gt) == 0:
        return 1.0
    dp = np.zeros((len(gt)+1, len(pred)+1))
    for i in range(len(gt)+1): dp[i][0] = i
    for j in range(len(pred)+1): dp[0][j] = j
    for i in range(1,len(gt)+1):
        for j in range(1,len(pred)+1):
            cost = 0 if gt[i-1]==pred[j-1] else 1
            dp[i][j] = min(dp[i-1][j]+1, dp[i][j-1]+1, dp[i-1][j-1]+cost)
    return dp[len(gt)][len(pred)] / len(gt)


In [10]:
for img_name, gt in tqdm(gt_data.items()):
    img = cv2.imread(os.path.join(IMG_DIR, img_name))
    if img is None:
        continue

    crop = crop_plate(img, gt["bbox"])
    out = reader.readtext(crop)

    pred = out[0][1] if out else ""
    pred = normalize_text(pred)

    c = cer(gt["plate_text"], pred)
    cer_values.append(c)

    if pred == gt["plate_text"]:
        exact += 1


  0%|          | 0/1337 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
  9%|▉         | 124/1337 [00:58<10:50,  1.87it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
100%|██████████| 1337/1337 [21:51<00:00,  1.02it/s]


In [11]:
print("Mean CER:", np.mean(cer_values))
print("Exact Match Acc:", exact / len(gt_data))


Mean CER: 0.423742807512441
Exact Match Acc: 0.12266267763649963
